In [77]:
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
from edc_metadata.constants import REQUIRED, KEYED
from django.apps import apps as django_apps
from django.db.models import Q
import pandas as pd
import re

In [78]:
intervention_df = pd.read_excel('./potlako_data_issues.xlsx', 
                                sheet_name='Intervention',
                                usecols='A,B,D',
                                engine='openpyxl',).dropna()




intervention_df

,PID,ISSUE,COMMENT
0,132-40400004-5,1000.1 amd 3000.1,1000.1 not on local host but on live server
1,132-40700015-8,"2000, 2000.1 AND 3000","2000 not on both servers, 2000.1 and 3000 not ..."
2,132-40700013-3,2000 AND 3000,"2000 not on both servers, 3000 not on live ser..."
3,132-40700014-1,2000 AND 3000,"2000 not on both servers, 3000 not on live ser..."
4,132-40500010-1,"2000, 2000.7, 3000","2000 only on local host, 2000.7 not on eitherm..."
5,132-40300005-3,2000 AND 3000,both visits not available on both servers
6,132-40300014-5,2000 AND 3000,both visits not available on both servers
7,132-40300007-9,2000 AND 3000,both visits not available on live servers
8,132-40300008-7,2000.1 and 3000,both visits not available on live servers
9,132-40300010-3,2000 AND 3000,both visits not available on live servers


In [79]:
standard_of_care_df = pd.read_excel('./potlako_data_issues.xlsx', 
                                    sheet_name='Standard of care',
                                    usecols='A,B,D',
                                    engine='openpyxl',).dropna()

standard_of_care_df

,PID,ISSUE,COMMENT
0,132-40200002-1,3000,not available on both servers
1,132-40200003-9,3000,not available on both servers
2,132-40200004-7,2000 and 3000,not available on both servers
3,132-40200006-2,2000 and 3000,not available on both servers
4,132-40200007-0,2000 and 3000,not available on both servers
5,132-40600013-4,3000,not available on both servers
6,132-40600015-9,3000,not available on both servers
7,132-40600033-2,2000 and 3000,not available on both servers
8,132-40600032-4,2000 and 3000,not available on both servers
9,132-40600031-6,2000 and 3000,not available on both servers


In [136]:
df = pd.concat([intervention_df, standard_of_care_df])

df

,PID,ISSUE,COMMENT
0,132-40400004-5,1000.1 amd 3000.1,1000.1 not on local host but on live server
1,132-40700015-8,"2000, 2000.1 AND 3000","2000 not on both servers, 2000.1 and 3000 not ..."
2,132-40700013-3,2000 AND 3000,"2000 not on both servers, 3000 not on live ser..."
3,132-40700014-1,2000 AND 3000,"2000 not on both servers, 3000 not on live ser..."
4,132-40500010-1,"2000, 2000.7, 3000","2000 only on local host, 2000.7 not on eitherm..."
...,...,...,...
41,132-40600034-0,2000 and 3000,not available on both servers
42,132-40600006-8,3000,not available on both servers
43,132-40600022-5,3000,not available on both servers
44,132-40900027-1,death report,not available on both servers


In [216]:
from edc_appointment.constants import NEW_APPT


regex = r"[0-9]{4}\.?[0-9]*"

missing_appointments = list()

for _, row in df.iterrows():
    

    visit_codes =  [str(float(visit_code)) for visit_code in re.findall(regex, str(row['ISSUE']))]
    subject_identifier = row['PID ']
    
    for visit_code in visit_codes:
        
        temp_visit_code = visit_code.split('.')[0]
        temp_visit_sequence = visit_code.split('.')[1]
        

        appointment = Appointment.objects.filter(subject_identifier=subject_identifier,
                                                 visit_code=temp_visit_code,
                                                 appt_status=NEW_APPT,
                                                 visit_code_sequence=temp_visit_sequence)
        
        if  appointment.exists():
            missing_appointments.extend(appointment.values_list('subject_identifier', 'appt_status', 'visit_code', 'visit_code_sequence'))
            
missing_appointments

        

[('132-40700015-8', 'new', '2000', 0),
 ('132-40700015-8', 'new', '2000', 1),
 ('132-40700015-8', 'new', '3000', 0),
 ('132-40700013-3', 'new', '2000', 0),
 ('132-40700013-3', 'new', '3000', 0),
 ('132-40700014-1', 'new', '2000', 0),
 ('132-40700014-1', 'new', '3000', 0),
 ('132-40500010-1', 'new', '2000', 0),
 ('132-40500010-1', 'new', '2000', 7),
 ('132-40300005-3', 'new', '2000', 0),
 ('132-40300005-3', 'new', '3000', 0),
 ('132-40300014-5', 'new', '2000', 0),
 ('132-40300014-5', 'new', '3000', 0),
 ('132-40300007-9', 'new', '2000', 0),
 ('132-40300007-9', 'new', '3000', 0),
 ('132-40300008-7', 'new', '2000', 1),
 ('132-40300008-7', 'new', '3000', 0),
 ('132-40300010-3', 'new', '2000', 0),
 ('132-40300010-3', 'new', '3000', 0),
 ('132-40300017-8', 'new', '2000', 0),
 ('132-40300017-8', 'new', '3000', 0),
 ('132-40700008-3', 'new', '2000', 1),
 ('132-40700008-3', 'new', '3000', 0),
 ('132-40300013-7', 'new', '1000', 2),
 ('132-40300013-7', 'new', '2000', 0),
 ('132-40500009-3', 'new'

In [217]:
appt= Appointment.objects.filter(subject_identifier='132-40500006-9', visit_code=3000).first()

In [218]:
from edc_appointment.constants import NEW_APPT


regex = r"[0-9]{4}\.?[0-9]*"

missing_visits = list()

for _, row in df.iterrows():
    

    visit_codes =  [str(float(visit_code)) for visit_code in re.findall(regex, str(row['ISSUE']))]
    subject_identifier = row['PID ']
    
    for visit_code in visit_codes:
        
        temp_visit_code = visit_code.split('.')[0]
        temp_visit_sequence = visit_code.split('.')[1]
        

        subject_visit = SubjectVisit.objects.filter(subject_identifier=subject_identifier,
                                                 visit_code=temp_visit_code,
                                                 visit_code_sequence=temp_visit_sequence)
        
        if  not subject_visit.exists():
            missing_visits.extend(subject_visit.values_list('subject_identifier', 'visit_code', 'visit_code_sequence'))
            
missing_visits

[]